In [ ]:
# helper method for col cleaning
years = np.linspace(2015, 2020, 6, dtype=int).tolist()
table4 = []
for y in years:
    n = y-2015
    table4_path='./Data_%s/CSV_files_%s/Table_4_Offenses_Offense_Type_by_Bias_Motivation_%s.csv'%(y,y,y)
    table4.append(pd.read_csv(table4_path, thousands=','))
table4[5]

base_col = table4[5].columns.tolist()
base_row = table4[5]['Bias_motivation'].tolist()
print(base_row)

i = 2015
for t in table4:
    for r in t['Bias_motivation']:
        if r not in base_row: print ('row: '+r,i)
    i += 1

j = 2015
for t in table4:
    for c in t.columns:
        if c not in base_col: print ('col: '+c,j)
    j += 1

In [ ]:
# regression
def reg(df, cat):
    X = df[['Year', cat]]
    y = df[['Total_offenses']]
    m1 = LinearRegression().fit(X,y)
    yhat = m1.predict(X)
    print('intercept: '+str(m1.intercept_))
    print('coefficient: '+str(m1.coef_))
    df[['Total_offenses']] = yhat
    ax = sns.scatterplot(data=df, x='Year', y='Total_offenses', hue=cat)
    sns.move_legend(ax, 'upper left', bbox_to_anchor=(1,1))
    plt.show()

def reg_plot(cats):
    cat_list = cats.to_list()
    cat_list = list(dict.fromkeys(cat_list))
    for cat in cat_list:
        cat_name = 'is_' + str(cat)
        reg(table_1, cat_name)

In [ ]:
# pie charts
def plot_pie(df):
    df = df.drop(['Bias_motivation', 'Total_offenses', 'Year', 'Category'],axis=1)

    data = df.iloc[2,:].to_numpy().tolist()
    labels = df.columns.to_list()
    colors = sns.color_palette('pastel')[0:15]
    fig, ax = plt.subplots()

    # plt.pie(data, labels = labels, colors = colors, autopct='%.1f%%')
    l = ax.pie(data, autopct='%.1f%%', startangle=-90)

    for label, t in zip(labels, l[1]):
        x, y = t.get_position()
        angle = int(math.degrees(math.atan2(y, x)))
        ha = "left"

        if x<0:
            angle -= 180
            ha = "right"

        plt.annotate(label, xy=(x,y), rotation=angle, ha=ha, va="center", rotation_mode="anchor", size=8)

    plt.show()
    
table_all.groupby('Year').apply(plot_pie)

In [ ]:
# wrong initial appraoch
%sql offense_type << SELECT * FROM table1_19 FULL OUTER JOIN table4_19 ON table1_19.Offenses = table4_19.Total_Offenses

offense_type = offense_type.loc[:,~offense_type.columns.duplicated()]
offense_type = offense_type.drop(['Total_offenses'],axis=1)
offense_type.head()

t4_19 = table4_19.drop(['Bias_motivation', 'Total_offenses'],axis=1)
sns.barplot(data=t4_19.iloc[1:2], orient = 'h')
plt.show()

# Simplify Table 4
%sql table4_16_type << SELECT Bias_motivation, Total_offenses AS '2016' FROM table4_16 WHERE Bias_motivation IN('Race/Ethnicity/Ancestry:', 'Religion:', 'Sexual Orientation:', 'Disability:', 'Gender:', 'Gender Identity:', 'Multiple-Bias Incidents')
%sql table4_17_type << SELECT Bias_motivation, Total_offenses AS '2017' FROM table4_17 WHERE Bias_motivation IN('Race/Ethnicity/Ancestry:', 'Religion:', 'Sexual Orientation:', 'Disability:', 'Gender:', 'Gender Identity:', 'Multiple-Bias Incidents')
%sql table4_18_type << SELECT Bias_motivation, Total_offenses AS '2018' FROM table4_18 WHERE Bias_motivation IN('Race/Ethnicity/Ancestry:', 'Religion:', 'Sexual Orientation:', 'Disability:', 'Gender:', 'Gender Identity:', 'Multiple-Bias Incidents')
%sql table4_19_type << SELECT Bias_motivation, Total_offenses AS '2019' FROM table4_19 WHERE Bias_motivation IN('Race/Ethnicity/Ancestry:', 'Religion:', 'Sexual Orientation:', 'Disability:', 'Gender:', 'Gender Identity:', 'Multiple-Bias Incidents')

# Combine Table 4 from 2016-2019
combined_table4 = pd.concat([table4_16_type, table4_17_type, table4_18_type, table4_19_type], axis=1)
combined_table4 = combined_table4.loc[:,~combined_table4.columns.duplicated()].copy()
combined_table4

tidy_table4['Year'] = tidy_table4['Year'].astype(int)
sns.lmplot(x='Year', y='Total_offenses', data=tidy_table4, hue='Bias_motivation')
plt.show()

def plot_by_bm(df):
    df['Year'] = df['Year'].astype(float)
    sns.lmplot(x='Year', y='Total_offenses', data=df, hue='Bias_motivation')
    plt.show()

tidy_table4.groupby(['Bias_motivation']).apply(plot_by_bm)

In [ ]:
# Plot output~one input groupy by State
def plot_single(df, input):
    sns.lmplot(x=input, y='Incidents',  col='State', data=df, hue='State')
    plt.show()

# Plot each input
for i in input:
    plot_single(table, i)